# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime
import numpy as np
tstart = '2017-11-10T00:00:00'
tend = '2017-11-11T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')

#tstart.strftime('%Y:%j:%H:%M:%S')


---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



../orbit_engine/lunar_314to315.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2017-11-10 01:29:53,2017-11-10 02:25:55
1,2017-11-10 03:06:32,2017-11-10 04:02:34
2,2017-11-10 04:43:11,2017-11-10 05:39:14
3,2017-11-10 06:19:50,2017-11-10 07:15:53
4,2017-11-10 07:56:30,2017-11-10 08:52:32


In [4]:
#from nustar_planning.lunar_planning import position, position_shift
import nustar_planning.lunar_planning as lunar_planning
from importlib import reload
reload(lunar_planning)

<module 'nustar_planning.lunar_planning' from '/Users/bwgref/anaconda/lib/python3.5/site-packages/nustar_planning-0.1.dev65-py3.5.egg/nustar_planning/lunar_planning.py'>

In [8]:
reload(lunar_planning)
sub = orbits.iloc[12:13].reset_index()
sub.head()
#lunar_planning.position(sub, steps = 5, parallax_correction=False)

,index,visible,occulted
0,12,2017-11-10 20:49:42,2017-11-10 21:45:46


In [10]:
# PA == 0 (default), Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)
reload(lunar_planning)
lunar_planning.position_shift(sub, parallax_correction=True, pa = 172.0*u.deg, dt = 3*u.s,
                              outfile='lunar_20171110_v1.txt', show=True)

2017:314:20:44:42 RA: 141.82302  Dec: 15.23017

2017:314:20:46:51 RA: 141.92384  Dec: 15.25913

2017:314:20:49:30 RA: 142.01405  Dec: 15.30888

2017:314:20:53:12 RA: 142.02531  Dec: 15.40828

2017:314:20:59:24 RA: 141.94564  Dec: 15.47342

2017:314:21:03:15 RA: 141.85188  Dec: 15.51836

2017:314:21:06:00 RA: 141.75282  Dec: 15.55349

2017:314:21:08:18 RA: 141.65099  Dec: 15.58208

2017:314:21:10:21 RA: 141.55002  Dec: 15.60512

2017:314:21:12:12 RA: 141.44525  Dec: 15.62459

2017:314:21:14:00 RA: 141.34198  Dec: 15.64001

2017:314:21:15:42 RA: 141.23607  Dec: 15.65228

2017:314:21:17:24 RA: 141.13223  Dec: 15.66097

2017:314:21:19:03 RA: 141.02572  Dec: 15.66645

2017:314:21:20:45 RA: 140.92126  Dec: 15.66830

2017:314:21:22:27 RA: 140.81750  Dec: 15.66634

2017:314:21:24:12 RA: 140.71377  Dec: 15.66000

2017:314:21:26:03 RA: 140.61059  Dec: 15.64829

2017:314:21:28:03 RA: 140.50820  Dec: 15.62927

2017:314:21:30:18 RA: 140.40805  Dec: 15.59882

2017:314:21:33:00 RA: 140.31916  Dec: 15